# The QR factorization

## Gram-Schmidt orthogonalization

The classical way to prove the existence of a QR factorization is to construct it via the Gram-Schmidt process. Let's look at it for a small example. 

In [22]:
A = round(10*rand(6,4))

6×4 Array{Float64,2}:
 6.0  7.0  8.0   2.0
 2.0  1.0  9.0  10.0
 4.0  7.0  5.0   5.0
 8.0  2.0  2.0   9.0
 9.0  7.0  3.0   1.0
 1.0  7.0  4.0   7.0

First we want an orthonormal basis for the span of the first column. That's pretty easy.

In [23]:
(Q,R) = (zeros(6,4),zeros(4,4));
R[1,1] = norm(A[:,1]);
Q[:,1] = A[:,1]/R[1,1];

Next we add another orthonormal vector to become a basis for the span of the first _two_ columns. We can do that via orthogonal projection...

In [24]:
P = Q[:,1]*Q[:,1]';  
v = A[:,2] - P*A[:,2];
@show v'*Q[:,1];

v' * Q[:,1] = [-4.44089e-16]


However, we'll organize the process in a particular way. 

$$
r_{22}\mathbf{q}_2 = \mathbf{v} = \mathbf{a}_2 - (QQ^*)\mathbf{a}_2 = \mathbf{a}_2 - Q(Q^*\mathbf{a}_2) = \mathbf{a}_2 - \mathbf{q}_1(r_{12})
$$

Doing so allows us to rearrange to

$$
\mathbf{a}_2 = \mathbf{q}_1r_{12} + \mathbf{q}_2 r_{22}, 
$$

which is the second column of $A=QR$. 

In [25]:
R[1,2] = dot(Q[:,1],A[:,2]);
v = A[:,2] - Q[:,1]*R[1,2];

R[2,2] = norm(v);
@show R;
Q[:,2] = v/R[2,2];
@show Q;

R = [14.2127 11.1168 0.0 0.0; 0.0 8.79863 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
Q = [0.422159 0.262192 0.0 0.0; 0.14072 -0.0641414 0.0 0.0; 0.281439 0.439988 0.0 0.0; 0.562878 -0.483874 0.0 0.0; 0.633238 -0.00450115 0.0 0.0; 0.0703598 0.706681 0.0 0.0]


Now the relevant projector is

In [26]:
P = Q[:,1:2]*Q[:,1:2]';
v = A[:,2] - P*A[:,2];
@show v'*Q[:,1:2];

v' * Q[:,1:2] = [-4.99936e-16 4.29766e-16]


But we break it into steps as before.

In [27]:
(R[1,3],R[2,3]) = dot(Q[:,1],A[:,3]),dot(Q[:,2],A[:,3]);
v = A[:,3] - Q[:,1:2]*R[1:2,3];
R[3,3] = norm(v);
@show R;
Q[:,3] = v/R[3,3];

R = [14.2127 11.1168 9.35785 0.0; 0.0 8.79863 5.56567 0.0; 0.0 0.0 8.96961 0.0; 0.0 0.0 0.0 0.0]


$Q$ is unitary and $R$ is upper triangular by construction, and we have

In [28]:
@show norm( Q*R[:,1:3] - A[:,1:3] );

norm(Q * R[:,1:3] - A[:,1:3]) = 1.039706991435451e-15


Rather than finish things manually, let's start over and automate everything in a loop.

In [29]:
(m,n) = size(A);
(Q,R) = (zeros(m,n),zeros(n,n));
R[1,1] = norm(A[:,1]);
Q[:,1] = A[:,1]/R[1,1];
for j = 2:n
    R[1:j-1,j] = Q[:,1:j-1]'*A[:,j];
    v = A[:,j] - Q[:,1:j-1]*R[1:j-1,j];
    R[j,j] = norm(v);
    Q[:,j] = v/R[j,j];
end

In [30]:
 @show Q;

Q = [0.422159 0.262192 0.288778 -0.549909; 0.14072 -0.0641414 0.896377 0.157757; 0.281439 0.439988 -0.00919676 0.119077; 0.562878 -0.483874 -0.0640208 0.556328; 0.633238 -0.00450115 -0.323391 -0.274042; 0.0703598 0.706681 -0.0659529 0.523387]


In [31]:
norm(Q'*Q-eye(n))

3.4692172871370876e-16

In [32]:
norm(A-Q*R)

1.039706991435451e-15

This only produces the "thin" version of the QR factorization. Conceptually at least, the full version just tacks on an orthonormal basis for whatever remains of $\mathbb{C}^m$.

## Linear system solution

We can apply QR to solve a linear system $A\mathbf{x}=\mathbf{b}$ for $m\times m$ matrix $A$. We have $QR\mathbf{x}=\mathbf{b}$, so $R\mathbf{x}=Q^*\mathbf{b}$ is an equivalent triangular system, solvable by back substitution. 

In [33]:
A = round(10*rand(9,9));
b = collect(1:9);
m = 9;
Q,R = qr(A);
z = Q'*b;
x = zeros(m);
x[m] = z[m]/R[m,m];
for i = m-1:-1:1
    y = z[i] - (R[i,i+1:m])'*x[i+1:m];
    x[i] = y[1] / R[i,i];
end
@show x;

x = [0.815123,0.565944,-0.0466339,-1.12679,-1.52542,0.751361,-0.913183,-0.515948,2.03705]


In [34]:
norm(A*x-b)

3.173917730451172e-14